In [2]:
import argparse
import collections
import time
import numpy as np
import pandas as pd
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as INIT
import torch.optim as optim
from torch.utils.data import DataLoader
import sys
sys.path.append('../')
from datasets import get_adjacency_matrix, get_locations
from utils import frac_type, prettyprint_args

import dgl
from dgl.data.tree import SSTDataset

from tree_lstm import TreeLSTM

In [13]:
def generate_batch(data, histlen=8):
    """Data is an unstacked dataframe -- index is timestamp, columns are
    monitor locations.
    """
    nodes = data.columns
    u, v = th.tensor(range(len(nodes))).repeat(len(nodes)),\
    th.tensor(np.array([[i]*len(nodes) for i in range(len(nodes))]).flatten())
    g = dgl.graph((u,v))
    n2t = lambda arr: th.from_numpy(np.array(arr)).float()
    for ii in range(data.shape[0]-histlen):
        batch = data.iloc[ii:ii+histlen+1,:].values
        if np.isfinite(batch).all():
            print('########')
            X = batch[:histlen,:].T
            Xt = [[step for step in node] for node in X]
            Xt = np.array(Xt)
            Xt = n2t(Xt).cuda()
            Y = batch[1:histlen+1,:]
            Yt = n2t(np.array([Y])).cuda()
            yield g, Xt, Yt

In [4]:
np.random.seed(0)
th.manual_seed(0)
th.cuda.manual_seed(0)

best_epoch = -1
best_dev_rmse = 1.0

cuda = 1
device = th.device('cuda') if cuda else th.device('cpu')
if cuda:
    th.cuda.set_device(-1)

fpath = '/scratch/ab9738/epod-nyu-delhi-pollution/data/govdata/govdata_1D_20180501_20201101.csv'
data = pd.read_csv(fpath, index_col=[0,1], parse_dates=True)['pm25']
data = data.unstack(level=0)
data.sort_index(axis=1, inplace=True)
data.drop('EastArjunNagar_CPCB', axis=1, inplace=True, errors='ignore')

data = data / 100.0
adj = get_adjacency_matrix('/scratch/ab9738/epod-nyu-delhi-pollution/data', thres=5000, n_max=None)

# training and validation sets
val_start_ind = int(0.8 * data.shape[0])
data_train = data.iloc[:val_start_ind,:]
data_val = data.iloc[val_start_ind:,:]

nodes = data.columns
res = data.index[1] - data.index[0]

models = []
for i in range(len(nodes)):
    model = TreeLSTM(len(nodes),
                     300,
                     150,
                     0.5).to(device)
    models.append(model)

print(models)
params_ex_emb =[x for x in list(model.parameters()) if x.requires_grad for model in models]

for p in params_ex_emb:
    if p.dim() > 1:
        INIT.xavier_uniform_(p)

optimizer = optim.Adagrad([
    {'params':params_ex_emb, 'lr':1e-3, 'weight_decay':1e-4}])

dur = []
criterion = nn.MSELoss(reduction='sum').cuda()

[TreeLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=150, out_features=1, bias=True)
  (cell): TreeLSTMCell(
    (W_iou): Linear(in_features=300, out_features=450, bias=False)
    (U_iou): Linear(in_features=300, out_features=450, bias=False)
    (U_f): Linear(in_features=300, out_features=300, bias=True)
  )
), TreeLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=150, out_features=1, bias=True)
  (cell): TreeLSTMCell(
    (W_iou): Linear(in_features=300, out_features=450, bias=False)
    (U_iou): Linear(in_features=300, out_features=450, bias=False)
    (U_f): Linear(in_features=300, out_features=300, bias=True)
  )
), TreeLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=150, out_features=1, bias=True)
  (cell): TreeLSTMCell(
    (W_iou): Linear(in_features=300, out_features=450, bias=False)
    (U_iou): Linear(in_features=300, out_features=450, bias=False)
    (U_f): Linear(in_features=300, 

/scratch/ab9738/epod-nyu-delhi-pollution/env/lib/python3.10/site-packages/torch/optim/adagrad.py:60: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adagrad, self).__init__(params, defaults)


In [18]:
for epoch in range(10):
    t_epoch = time.time()
    for step, batch in enumerate(generate_batch(data_train)):
        g = batch[0].to(device)
        n = g.number_of_nodes()
        h = th.zeros((n, 150)).to(device)
        c = th.zeros((n, 150)).to(device)
        if step >= 3:
            t0 = time.time() # tik

        print(batch[1].shape, batch[2].shape)
        for q, model in enumerate(models):
            preds = model(batch[1], g, h, c)
            # each node has its own model, which is backpropagated through that node's predictions
            loss = criterion(preds, batch[2][:, q], reduction='sum')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if step >= 3:
                dur.append(time.time() - t0) # tok

            if step > 0 and step % args.log_every == 0:
                print("Epoch {:05d} | Step {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
                    epoch, step, loss.item(), np.mean(dur)))
    print('Epoch {:05d} training time {:.4f}s'.format(epoch, time.time() - t_epoch))

    rmse = []
    for step, batch in enumerate(generate_batch(data_val)):
        print('######')
        g = batch[0].to(device)
        n = g.number_of_nodes()
        with th.no_grad():
            print('######')
            h = th.zeros((n, args.h_size)).to(device)
            c = th.zeros((n, args.h_size)).to(device)
            for q, model in enumerate(models):
                preds = model(batch[1], g, h, c)
                loss = criterion(preds, batch[2][:, q], reduction='sum')
                print('######')
                rmse.append(loss.item())

    dev_rmse = 1.0*np.sum([x for x in rmse])/len(rmse)
    print("Epoch {:05d} | Loss {:.4f}".format(epoch, dev_rmse))

    if dev_rmse > best_dev_rmse:
        best_dev_rmse = dev_rmse
        best_epoch = epoch
        for q, model in enumerate(models):
            th.save(model.state_dict(), 'best_{}_{}.pkl'.format(args.seed, q))
    else:
        if best_epoch <= epoch - 10:
            break

    # lr decay
    for param_group in optimizer.param_groups:
        param_group['lr'] = max(1e-5, param_group['lr']*0.99) #10
        print(param_group['lr'])

########
torch.Size([32, 8]) torch.Size([1, 8, 32])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x8 and 300x450)

In [87]:
dgl.__version__

'0.8.1'

In [91]:
import dgl; print(dgl.__path__)

['/scratch/ab9738/epod-nyu-delhi-pollution/env/lib/python3.10/site-packages/dgl']
